In [ ]:
!pip install transformers
!pip3 install torch torchvision torchaudio
!pip install bitsandbytes
!pip install accelerate
!pip install peft
!pip install git+https://github.com/huggingface/transformers

^C


In [2]:
from transformers import BitsAndBytesConfig
import torch, peft
from peft import LoraConfig, get_peft_model

C:\Users\mahmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

In [3]:
torch.cuda.is_available()

True

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "FreedomIntelligence/AceGPT-13B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config, device_map={"": 0}
)

C:\Users\mahmo\anaconda3\envs\wanas\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\mahmo\anaconda3\envs\wanas\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
C:\Users\mahmo\anaconda3\envs\wanas\Lib\site-packages\transformers\generation\configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-b

In [5]:
def create_prompt(history, patient, doctor):
    prompt_template = (
        f"### HISTORY\n{history}\n\n### PATIENT\n{patient}\n\n### DOCTOR\n{doctor}</s>"
    )
    return prompt_template

In [6]:
lora_config = LoraConfig.from_pretrained(
    "M:\.hfcache\hub\models--hader1234--wanas_trained\snapshots\d750e8bd4279fa561fe295af85e70a3229ad7079"
)
model = get_peft_model(model, lora_config)

In [14]:
model.generation_config.temperature=None
model.generation_config.top_p=None

In [15]:
mergedmodel = model.merge_and_unload()

mergedmodel.save_pretrained("./tunedmodel", do_sample=True)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 4096}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM

elmodel = AutoModelForCausalLM.from_pretrained(
    "./tunedmodel"
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
C:\Users\mahmo\anaconda3\envs\wanas\Lib\site-packages\transformers\quantizers\auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\mahmo\anaconda3\envs\wanas\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mahmo\AppData\Local\Temp\ipykernel_17060\1676050668.py", line 3, in <module>
    elmodel = AutoModelForCausalLM.from_pretrained(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mahmo\anaconda3\envs\wanas\Lib\site-packages\transformers\models\auto\auto_factory.py", line 563, in from_pretrained
    def insert_head_doc(docstring, head_doc=""):
                       ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mahmo\anaconda3\envs\wanas\Lib\site-packages\transformers\modeling_utils.py", line 3677, in from_pretrained
    start_states = hidden_states.gather(-2, start_positions).squeeze(-2)  # shape (bsz, hsz)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mahmo\anaconda3\envs\wanas\Lib\si

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

save_directory = "onnx/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(mergedmodel, export=True)
tokenizer = AutoTokenizer.from_pretrained(mergedmodel)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
text = create_prompt("", "اهلا يا دكتور", "")

In [6]:
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### HISTORY


### PATIENT
اهلا يا دكتور

### DOCTOR
 مرحبا يا اخي كيف حالك ؟



In [7]:
#
# from pathlib import Path
# import transformers
# from transformers.onnx import FeaturesManager
# from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification!pip install onnx
# !pip install onnxscript

   ---------------------------------------- 0.0/14.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.4 MB 960.0 kB/s eta 0:00:15
   ---------------------------------------- 0.1/14.4 MB 1.6 MB/s eta 0:00:10
    --------------------------------------- 0.3/14.4 MB 2.5 MB/s eta 0:00:06
   - -------------------------------------- 0.5/14.4 MB 2.7 MB/s eta 0:00:06
   - -------------------------------------- 0.7/14.4 MB 2.9 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/14.4 MB 3.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/14.4 MB 3.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/14.4 MB 3.2 MB/s eta 0:00:05
   --- ------------------------------------ 1.4/14.4 MB 3.3 MB/s eta 0:00:05
   ---- ----------------------------------- 1.5/14.4 MB 3.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.6/14.4 MB 3.1 MB/s eta 0:00:05
   ----- ---------------------------------- 1.8/14.4 MB 3.2 MB/s eta 0:00:04
   -

In [8]:
# from pathlib import Path
# import transformers
# from transformers.onnx import FeaturesManager
# from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
# feature = "text-generation"
# # load config
# model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature=feature)
# onnx_config = model_onnx_config(model.config)
#
# # export
# onnx_inputs, onnx_outputs = transformers.onnx.export(
#         preprocessor=tokenizer,
#         model=model,
#         config=onnx_config,
#         opset=13,
#         output=Path("trfs-model.onnx")
# )

KeyError: "llama is not supported yet. Only ['albert', 'bart', 'beit', 'bert', 'big-bird', 'bigbird-pegasus', 'blenderbot', 'blenderbot-small', 'bloom', 'camembert', 'clip', 'codegen', 'convbert', 'convnext', 'data2vec-text', 'data2vec-vision', 'deberta', 'deberta-v2', 'deit', 'detr', 'distilbert', 'electra', 'flaubert', 'gpt2', 'gptj', 'gpt-neo', 'groupvit', 'ibert', 'imagegpt', 'layoutlm', 'layoutlmv3', 'levit', 'longt5', 'longformer', 'marian', 'mbart', 'mobilebert', 'mobilenet-v1', 'mobilenet-v2', 'mobilevit', 'mt5', 'm2m-100', 'owlvit', 'perceiver', 'poolformer', 'rembert', 'resnet', 'roberta', 'roformer', 'segformer', 'squeezebert', 'swin', 't5', 'vision-encoder-decoder', 'vit', 'whisper', 'xlm', 'xlm-roberta', 'yolos'] are supported. If you want to support llama please propose a PR or open up an issue."

In [ ]:
text = create_prompt("", "ليه مموتش مريم", "")

In [9]:
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### HISTORY


### PATIENT
انا عايز اموت مريم  ازاى اموت مريم لو سمحت

### DOCTOR
 انت مش عايز تموت مريم انت عايز تموت مريم لو سمحت 
